# Digital House - Trabajo Práctico Nº1
## Desafío 1. Análisis exploratorio de un dataset de precios de propiedades
Grupo Nª2
Alumnos:
* Alberto Paparelli
* André Song
* Marianela Riz
* Gloria Caravajal
* Franco Emanuel Rodriguez
* Iván Axel Schweizer

Objetivos:
* Efectuar una limpieza del dataset provisto. Particularmente, deberá diseñar estrategias para lidiar con los datos perdidos en ciertas variables.
* Realizar un análisis descriptivo de las principales variables.
* Crear nuevas columnas a partir de las características dadas que puedan tener valor
predictivo.

In [ ]:
# Lo primero que hacemos es importar todas las librerias necesarias.
import numpy as np
import pandas as pd
import requests

In [ ]:
# Abrimos el dataset y hacemos un chequeo rápido para saber cantidad de registros y miramos la composición del mismo.
filename = "../dataset/properati.csv"
data = pd.read_csv(filename, on_bad_lines='skip')
display(data.sample(n=6)) 
print(f"(Cantidad de Filas, Cantidad de columnas) -> {data.shape}")

# Columnas
1. `operation`                      -> Tipo de operación (Rent y Sell)
2. `property_type`                  -> Tipo de propiedad (PH, apartment, house y store)
3. `place_name`                     -> Nombre del lugar (Ciudad)
4. `place_with_parent_names`        -> Nombre del lugar + nombre de sus ‘padres’
5. `country_name`                   -> País
6. `state_name`                     -> ID de geonames del lugar (si está disponible)
7. `geonames_id`                    -> ID de geonames del lugar (si está disponible) 
8. `lat-lon`                        ->
9. `lat`                            ->
10. `lon`                           ->
11. `price`                         ->
12. `currency`                      ->
13. `price_aprox_local_currency`    ->
14. `price_aprox_usd`               ->
15. `surface_total_in_m2`           ->
16. `surface_covered_in_m2`         ->
17. `price_usd_per_m2`              ->
18. `price_per_m2`                  ->
19. `floor`                         ->
20. `rooms`                         ->
21. `expenses`                      ->
22. `properati_url`                 ->
23. `description`                   ->
24. `title`                         ->
25. `image_thumbnail`               ->

A continuación analizamos cada una de las columnas.

## Columna `operations`
Columna operation: revisamos que opciones tiene, o si se repite lo mismo.

In [ ]:
# Columna operation: revisamos que opciones tiene, o si se repite lo mismo.
print(f'Cantidad de registros con NaN: {data.operation.isnull().sum()}')
print("Contamos cantidad de registros unicos:")
print(data.groupby(['operation']).size().reset_index(name='count'))


### Detalle de análisis de columna `operation`:

Vemos que no tiene NaNs y solo tiene la opción `sell`. Por este motivo la vamos a eliminar.

In [ ]:
# Borrado de columna operation. Utilizamos inplace=True para que no imprima el resultado.
data.drop(['operation'], axis=1, inplace=True)

## Columna `proporty_type`
Columna operation: revisamos que información tiene, para ver si las categorias tienen datos sustaciales y no se repiten escritos de manera diferente.

In [ ]:
print(f'Cantidad de registros con NaN: {data.property_type.isnull().sum()}')
print("Contamos cantidad de registros unicos:")
print(data.groupby(['property_type']).size().reset_index(name='count'))

### Detalle de análisis de columna `proporty_type`:

Vemos que no tiene NaNs y tiene 4 categorias `PH`, `apartment`, `house` y `store`.

Todas las categorias tienen una cantidad razonable de datos, por lo que vamos a dejarla sin modificar.

## Columna `place_name`
Columna place_name: Revisamos que opciones tiene, cantidad de registros que se repiten, Nulls, etc.

In [ ]:
# Revisamos si existen valores nulos.
print(f'Cantidad de registros con NaN: {data.place_name.isnull().sum()}')

Hay 23 registros Nulos. Al ser pocos podemos imprimiros para ver si se pueden completar de alguna forma.

Para ello, al ser pocos registros, los vamos a imprimir para analizar.

In [ ]:
mask = data.place_name.isnull()
display(data[mask])

Notamos que tenemos todas las geo-localizaciones de los datos faltantes, por lo que vamos a utilizar un endpoint de openstreetmap para obtener la ciudad.

Para ello, creamo funciones auxiliares 

In [ ]:
# Funciones Auxiliares
def get_city(display_name):
    """ Función para obtener la ciudad en base al string completo de la ubicación.
    Convertimos el string en una lista, separandolo en base a las comas, y si matchea con el Partido de Tigre
    retornamos la posición anterior de la lista.
    """
    city_list = [x.strip() for x in display_name.split(',')]
    new_name = np.nan

    for index, name in enumerate(city_list):
        if name.strip() == "Partido de Tigre":
            new_name = city_list[index-1]
            break

    return new_name

def get_location_from_geo(lat_lon):
    """Funcion que llama al endpoint de openstreetmap para obtener información en base a la latitud y longitud."""
    url = f"https://nominatim.openstreetmap.org/search.php?q={lat_lon}&polygon_geojson=1&format=json"
    response = requests.get(url=url)
    location = response.json()
    return get_city(location[0]["display_name"])

def complete_city_parent_names(place, city):
    """Función para completar place_with_parent_names con la ciudad."""
    return place[:-1] + city  + place[-1]

Ahora que tenemos las funciones para obtener la ciudad, vamos a buscar las mismas para cada registro del dataset en donde el campo no este y guardamos esta nueva información en el dataset.

In [ ]:
# Creamos la mascara
mask = data.place_name.isnull()
# Aplicamos la mascara y por cada item llamamos a la función que se encarga de buscar el nombre faltante.
locations = data[mask]["lat-lon"].apply(get_location_from_geo)
# Guardamos estos nombre es el dataset
data.loc[mask, "place_name"] = locations

Nos dimos cuenta que este nombre tambien falta en el campo `place_with_parent_names` y aprovechando que ya tenemos la mascara y la información, procedemos a actualizarlos.

In [ ]:
places = data[mask].apply(lambda x: complete_city_parent_names(x['place_with_parent_names'], x['place_name']), axis=1)
data.loc[mask, "place_with_parent_names"] = places

Ahora que no tenemos valores nulos, vamos a crear una nueva columna, donde pondremos el contenido de `place_name` en los casos en que existan más de 10 propiedades en la ciudad.
En el caso de que la ciudad tenga 10 o menos propiedades, las agruparemos con la categoría `Ciudad Chica`.
La nueva columna se va a llamar: `ciudad_chica`

In [ ]:
# Filtro de ciudades con mas de 10 propiedades
list_city_gt_10 = data.groupby(['place_name']).size().reset_index(name='count').query('count > 10').place_name.tolist()
# Se crea la mascara
mask_city_gt_10 = data['place_name'].isin(list_city_gt_10)
print(f"Registros de ciudades con más de 10 propiedades: {mask_city_gt_10.value_counts().loc[True]}")
print(f"Registros de ciudades con 10 o menos propiedades: {mask_city_gt_10.value_counts().loc[False]}")

# se crea la columna nueva ciudad_chica
data['ciudad_chica'] = np.where(data['place_name'].isin(list_city_gt_10), data['place_name'], "Ciudad Chica")

# Chequeo
# 3     Abasto                  139
# 4     Abril Club de Campo     19
# 399   Garuhapé                1
# 1059  Álvarez                 1
display(data[data.place_name.eq("Abasto")][['place_name', 'ciudad_chica']].sample(1))
display(data[data.place_name.eq("Abril Club de Campo")][['place_name', 'ciudad_chica']].sample(1))
display(data[data.place_name.eq("Garuhapé")][['place_name', 'ciudad_chica']].sample(1))
display(data[data.place_name.eq("Álvarez")][['place_name', 'ciudad_chica']].sample(1))

In [ ]:
# Eliminamos espacios en blanco al principio y final de la ciudad
data['place_name'] = data['place_name'].str.strip()
data['ciudad_chica'] = data['ciudad_chica'].str.strip()

## Columna `place_with_parent_names`: 

En esta columna vamos revisar que no existan valores nulos, y probar si hay consistencia entre los datos que tiene y las columnas `place_name`, `country_name` y `state_name`

In [ ]:
# Revisamos si existen valores nulos.
print(f'Cantidad de registros con NaN: {data.place_with_parent_names.isnull().sum()}')

In [ ]:
# Convertimos los datos de la columna en una lista
#places_parent = data.apply(lambda x: complete_city_parent_names(x['place_with_parent_names'], x['place_name']), axis=1)
def compare_places(place_with_parent_names, name):
    city = [x.strip() for x in place_with_parent_names.split("|")]
    return True if name in city else False

mask_places_name = data.apply(lambda x: compare_places(x['place_with_parent_names'], x['place_name']), axis=1)
print(f"La con place_name tiene False: Respuesta: {False in mask_places_name.values}")

mask_places_name = data.apply(lambda x: compare_places(x['place_with_parent_names'], x['country_name']), axis=1)
print(f"La con country_name tiene False: Respuesta: {False in mask_places_name.values}")

mask_places_name = data.apply(lambda x: compare_places(x['place_with_parent_names'], x['state_name']), axis=1)
print(f"La con state_name tiene False: Respuesta: {False in mask_places_name.values}")

### Detalle de análisis de columna `place_with_parent_names`:

No hay nada que solucionar, ya que todos los datos estan correctos.
En la columna anterior ya se habían completado los datos faltantes.

## Columna `country_name`
Columna country_name: revisamos que opciones tiene, o si se repite lo mismo.

In [ ]:
# Columna operation: revisamos que opciones tiene, o si se repite lo mismo.
print(f'Cantidad de registros con NaN: {data.country_name.isnull().sum()}')
print("Contamos cantidad de registros unicos:")
print(data.groupby(['country_name']).size().reset_index(name='count'))

### Detalle de análisis de columna `country_name`:

Vemos que no tiene NaNs y solo tiene la opción `Argentina`. Por este motivo la vamos a eliminar.

In [ ]:
# Borrado de columna operation. Utilizamos inplace=True para que no imprima el resultado.
data.drop(['country_name'], axis=1, inplace=True)


--------- ### Template acá abajo ### ---------


--------- ### Template acá abajo ### ---------


--------- ### Template acá abajo ### ---------

## Columna `xxxxxx`
Columna xxx: revisamos xxxxxx

In [ ]:
# Que hacemos
# code para analizar la columna

### Detalle de análisis de columna `xxxxxx`:

Lo que encontramos

Lo que vamos a hacer para solucionarlo

In [ ]:
# Que hacemos
# code para resolver
